> https://courses.engr.illinois.edu/cs225/fa2023/resources/stack-heap/

# 1 Overview
When a program is running, it takes up memory. Sometimes we are not even aware of the memory being allocated. In fact, every time you create a new variable, your program is allocating more memory for you to store that variable. This article focuses on two kinds of memories: stack and heap.

# 2 General Memory Layout
Each running program has its own memory layout, separated from other programs. The layout consists of a lot of segments, including:

stack: stores local variables
heap: dynamic memory for programmer to allocate
data: stores global variables, separated into initialized and uninitialized
text: stores the code being executed
In order to pinpoint each memory location in a program’s memory, we assign each byte of memory an “address”. The addresses go from 0 all the way to the largest possible address, depending on the machine. As the figure below, the text, data, and heap segments have low address numbers, while the stack memory has higher addresses.

![](./images/memory_layout.jpeg)

By convention, we express these addresses in base 16 numbers. For instance, the smallest possible address is 0x00000000 (where the 0x means base 16), and the largest possible address could be 0xFFFFFFFF.

# 3 allocate stack memory for a local variable in the scope of a function

As shown above, the stack segment is near the top of memory with high address. Every time a function is called, the machine allocates some stack memory for it. When a new local variables is declared, more stack memory is allocated for that function to store the variable. Such allocations make the stack grow downwards. After the function returns, the stack memory of this function is deallocated, which means all local variables become invalid. The allocation and deallocation for stack memory is automatically done. The variables allocated on the stack are called stack variables, or automatic variables.

The following figures show examples of what stack memory looks like when the corresponding code is run:

![allocate_memory_for_local_variable_in_stack](./images/allocate_memory_for_local_variable_in_stack.jpeg)

## common mistake: return a pointer from the address of a local vaibale in low-level function

Since the stack memory of a function gets deallocated after the function returns, there is no guarantee that the value stored in those area will stay the same. A common mistake is to return a pointer to a stack variable in a helper function. After the caller gets this pointer, the invalid stack memory can be overwritten at anytime. The following figures demonstrate one example of such scenario. Assume there is a Cube class that has methods getVolume and getSurfaceArea, as well as a private variable width.

![allocate_memory_for_local_variable_in_stack](./images/common_mistake.jpeg)

# 4 allocate heap memory for a object with controllable lifetime by programmer

In the previous section we saw that functions cannot return pointers of stack variables. To solve this issue, you can either return by copy, or put the value at somewhere more permanent than stack memory. Heap memory is such a place. Unlike stack memory, heap memory is allocated explicitly by programmers and it won’t be deallocated until it is explicitly freed. To allocate heap memory in C++, use the keyword new followed by the constructor of what you want to allocate. The return value of new operator will be the address of what you just created (which points to somewhere in the heap).

The figures below demonstrate what happens in both stack and heap when the corresponding code is executed:

## 4.1 The need for dynamic memory allocation

> https://www.learncpp.com/cpp-tutorial/dynamic-memory-allocation-with-new-and-delete/

![allocate_memory_for_local_variable_in_stack](./images/allocate_memory_in_heap.jpeg)

## 4.2 how and what happens when stack-heap memory works?

To free heap memory, use the key word delete followed by the pointer to the heap memory. Be careful about the memory you freed. If you try to use the pointers to those memory after you free them, it will cause undefined behavior. To avoid such issues, it is good practice to set the value of freed pointers to nullptr immediately after delete. Here is an example that correctly frees memory after using it.

![](./images/step1.jpeg)

![](./images/step2.jpeg)

![](./images/step3.jpeg)

![](./images/step4.jpeg)

![](./images/step5.jpeg)

![](./images/step6.jpeg)

![](./images/step7.jpeg)

![](./images/step8.jpeg)

> In the figures above, you can see that heap memory are not allocated continuously from bottom to top. This is because unlike stack where the invalid memory is always at the bottom, the user can free heap memory that’s in between valid memories, causing fragmentations in the heap. In order to reuse memory efficiently, there are numerous heap allocation scheme that try to pick the “best” spot for you. You will learn more about memory allocation in a system programming class.

## 4.3 what mistakes will happen when use heap-memory

### 4.3.1 memory leaks

> https://www.learncpp.com/cpp-tutorial/dynamic-memory-allocation-with-new-and-delete/

Consider the following function:
~~~
void doSomething()
{
    int* ptr{ new int{} };
}
~~~

Dynamically allocated memory stays allocated until it is explicitly deallocated or until the program ends (and the operating system cleans it up, assuming your operating system does that). However, the pointers used to hold dynamically allocated memory addresses follow the normal scoping rules for local variables. This mismatch can create interesting problems.

This function allocates an integer dynamically, but never frees it using delete. **Because pointers variables are just normal variables, when the function ends, ptr will go out of scope. And because ptr is the only variable holding the address of the dynamically allocated integer, when ptr is destroyed there are no more references to the dynamically allocated memory. This means the program has now “lost” the address of the dynamically allocated memory. As a result, this dynamically allocated integer can not be deleted.**

This is called a memory leak. Memory leaks happen when your program loses the address of some bit of dynamically allocated memory before giving it back to the operating system. When this happens, your program can’t delete the dynamically allocated memory, because it no longer knows where it is. The operating system also can’t use this memory, because that memory is considered to be still in use by your program.

Memory leaks eat up free memory while the program is running, making less memory available not only to this program, but to other programs as well. Programs with severe memory leak problems can eat all the available memory, causing the entire machine to run slowly or even crash. Only after your program terminates is the operating system able to clean up and “reclaim” all leaked memory.

Although memory leaks can result from a pointer going out of scope, there are other ways that memory leaks can result. For example, a memory leak can occur if a pointer holding the address of the dynamically allocated memory is assigned another value:

### 4.3.2 Dangling pointers

C++ does not make any guarantees about what will happen to the contents of deallocated memory, or to the value of the pointer being deleted. In most cases, the memory returned to the operating system will contain the same values it had before it was returned, and the pointer will be left pointing to the now deallocated memory.

A pointer that is pointing to deallocated memory is called a dangling pointer. Dereferencing or deleting a dangling pointer will lead to undefined behavior. Consider the following program:

~~~
#include <iostream>

int main()
{
    int* ptr{ new int }; // dynamically allocate an integer
    *ptr = 7; // put a value in that memory location

    delete ptr; // return the memory to the operating system.  ptr is now a dangling pointer.

    std::cout << *ptr; // Dereferencing a dangling pointer will cause undefined behavior
    delete ptr; // trying to deallocate the memory again will also lead to undefined behavior.

    return 0;
}
~~~

In the above program, the value of 7 that was previously assigned to the allocated memory will probably still be there, but it’s possible that the value at that memory address could have changed. It’s also possible the memory could be allocated to another application (or for the operating system’s own usage), and trying to access that memory will cause the operating system to shut the program down.

Deallocating memory may create multiple dangling pointers. Consider the following example:
~~~
#include <iostream>

int main()
{
    int* ptr{ new int{} }; // dynamically allocate an integer
    int* otherPtr{ ptr }; // otherPtr is now pointed at that same memory location

    delete ptr; // return the memory to the operating system.  ptr and otherPtr are now dangling pointers.
    ptr = nullptr; // ptr is now a nullptr

    // however, otherPtr is still a dangling pointer!

    return 0;
}
~~~

There are a few best practices that can help here.
    
* First, try to avoid having multiple pointers point at the same piece of dynamic memory. If this is not possible, be clear about which pointer “owns” the memory (and is responsible for deleting it) and which pointers are just accessing it.

* Second, when you delete a pointer, if that pointer is not going out of scope immediately afterward, set the pointer to nullptr. We’ll talk more about null pointers, and why they are useful in a bit.

# 5 conclusion

here is the key points:
* A pointer variable (or pointer in short) is basically the same as the other variables, which can store a piece of data. Unlike normal variable which stores a value (such as an int, a double, a char), a pointer stores a memory address. that's saying pointer is an object that holds a memory address (typically of another variable) as its value. This allows us to store the address of some other object to use later.

* the raw pointer own the ownership of the object in the heap that points to, the meanning of ownership here is: the pointer has the right to control the lifetime of that object in heap, by creation(new) and destroy(delete ptr)

* A pointer is a local variable in the scope of a function, when the function ends, ptr will go out of scope. And because ptr is the only variable holding the address of the dynamically allocated integer, when ptr is destroyed there are no more references to the dynamically allocated memory. This means the program has now “lost” the address of the dynamically allocated memory. As a result, this dynamically allocated integer can not be deleted. this will result in memory leaks.

### what to do

* First, try to avoid having multiple pointers point at the same piece of dynamic memory. If this is not possible, be clear about which pointer “owns” the memory (and is responsible for deleting it) and which pointers are just accessing it.

* Second, when you delete a pointer, if that pointer is not going out of scope immediately afterward, set the pointer to nullptr. We’ll talk more about null pointers, and why they are useful in a bit.